In [22]:
# Dependencies
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import scipy.stats as st
from scipy.stats import linregress
from pandas.core.reshape.pivot import pivot


In [23]:
# Read CSV
df1 = pd.read_csv("raw_data/2017-18-vacancy.csv")
df2 = pd.read_csv("raw_data/2018-19-vacancy.csv")
df3 = pd.read_csv("raw_data/2019-20-vacancy.csv")
df4 = pd.read_csv("raw_data/2020-21-vacancy.csv")

In [24]:
df1.columns

Index(['Postcode', 'Bedrooms', 'RentalUnitDesc', 'VUDate', 'VTDate', 'TENDate',
       'RUUseType', 'HousingServiceCentre', 'VUDays', 'VTDays', 'TotalVAC',
       'LGA', 'StateElectorate'],
      dtype='object')

In [25]:
df2.columns

Index(['Postcode', 'Bedrooms', 'RentalUnitDesc', 'VUDate', 'VTDate', 'TENDate',
       'RUUseType', 'HousingServiceCentre', 'VUDays', 'VTDays', 'TotalVAC',
       'LGA', 'StateElectorate'],
      dtype='object')

In [29]:
#Fixing column names and reordering as needed
df3 = df3.rename(columns= {'PropertyType':'RentalUnitDesc', 'TotalVacDays':'TotalVac', 'LocalGovtAuthority':'LGA'})
neworder = ['Postcode', 'Bedrooms', 'RentalUnitDesc', 'VUDate', 'VTDate', 'TENDate',
       'RUUseType', 'HousingServiceCentre', 'VUDays', 'VTDays', 'TotalVAC',
       'LGA', 'StateElectorate']
df3=df3.reindex(columns=neworder)
df3.columns

Index(['Postcode', 'Bedrooms', 'RentalUnitDesc', 'VUDate', 'VTDate', 'TENDate',
       'RUUseType', 'HousingServiceCentre', 'VUDays', 'VTDays', 'TotalVAC',
       'LGA', 'StateElectorate'],
      dtype='object')

In [31]:
#Fixing column names and reordering as needed
df4 = df4.rename(columns= {'PropertyType':'RentalUnitDesc', 'LocalGovtAuthority':'LGA'})
df4=df4.reindex(columns=neworder)
df4.columns

Index(['Postcode', 'Bedrooms', 'RentalUnitDesc', 'VUDate', 'VTDate', 'TENDate',
       'RUUseType', 'HousingServiceCentre', 'VUDays', 'VTDays', 'TotalVAC',
       'LGA', 'StateElectorate'],
      dtype='object')

In [32]:
df4.tail()

,Postcode,Bedrooms,RentalUnitDesc,VUDate,VTDate,TENDate,RUUseType,HousingServiceCentre,VUDays,VTDays,TotalVAC,LGA,StateElectorate
6262,4101,1,Apartment,1/08/2020,1/09/2020,1/09/2020,Community Housing,FORTITUDE VALLEY,23,0,23,Brisbane,South Brisbane
6263,4034,3,Apartment,1/05/2021,1/06/2021,1/06/2021,Community Housing,CHERMSIDE,28,0,28,Brisbane,Nudgee
6264,4101,1,Apartment,1/07/2020,1/08/2020,1/08/2020,Community Housing,FORTITUDE VALLEY,33,0,33,Brisbane,South Brisbane
6265,4020,1,Apartment,1/12/2020,1/01/2021,1/02/2021,Community Housing,MORETON BAY,35,18,53,Moreton Bay,Redcliffe
6266,4020,1,Apartment,1/03/2021,1/04/2021,1/04/2021,Community Housing,MORETON BAY,46,0,46,Moreton Bay,Redcliffe


In [33]:
#Concatenating all dataframes
df = pd.concat([df1, df2, df3, df4])
#Dropping nan values
df = df.dropna(subset=['TotalVAC'],inplace = False)
#Dropping rows with 0 days vacant
df = df[df.TotalVAC != 0]
df.shape

(28103, 13)

In [34]:
#Create two new columns for year and month
Years = []
Months = []
#Convert date string to year and month
date_splited = []
dates = pd.Series(df['TENDate'])
for date in dates:
    if '/' in date:
        date_splited = date.split('/')
        Years.append(date_splited[2].strip())
        Months.append(f'{date_splited[2]}-{date_splited[1]}')
    elif '-' in date:
        date_splited = date.split('-')
        Years.append(f'20{date_splited[1].strip()}')
        Months.append(date_splited[0].strip())
    else:
        Years.append(date)

print(len(dates))
print(len(Years))
df['Year'] = Years
df['Month'] = Months

28103
28103


In [35]:
#Check rental unit description's case
df['RentalUnitDesc'].unique()

array(['DETACHED HOUSE', 'APARTMENT', 'SENIOR UNIT', 'DUPLEX',
       'ATTACHED HOUSING', 'CLUSTER HOUSING', 'DUAL OCCUPANCY',
       'Townhouse', 'Detached House', 'Apartment', 'Cluster House',
       'Duplex', 'Room', 'Dual Occupancy'], dtype=object)

In [36]:
#Realign the case type of RentalUnitDesc
#Convert Upper case to title case
for index, row in df.iterrows():
    rentalunit = row['RentalUnitDesc']
    #print(rentalunit)
    if rentalunit.isupper():
        df.loc[index, 'RentalUnitDesc'] = rentalunit.title()

df['RentalUnitDesc'].unique()
    

array(['Detached House', 'Senior Unit', 'Attached Housing', 'Apartment',
       'Cluster Housing', 'Duplex', 'Dual Occupancy', 'Townhouse'],
      dtype=object)

In [37]:
#Write to CSV
df.to_csv("all_vacancy_data.csv")
df.head()

,Postcode,Bedrooms,RentalUnitDesc,VUDate,VTDate,TENDate,RUUseType,HousingServiceCentre,VUDays,VTDays,TotalVAC,LGA,StateElectorate,Year,Month
0,4032,3,Detached House,9/10/2017,2/11/2017,3/11/2017,Public Housing,105,24,1,25,BRISBANE,STAFFORD,2017,2017-11
1,4108,2,Detached House,3/04/2018,18/04/2018,2/05/2018,Public Housing,107,15,14,29,BRISBANE,TOOHEY,2018,2018-05
2,4108,2,Detached House,1/08/2017,11/08/2017,21/08/2017,Public Housing,107,10,10,20,BRISBANE,TOOHEY,2017,2017-08
3,4108,2,Detached House,1/03/2018,23/03/2018,28/03/2018,Public Housing,107,22,5,27,BRISBANE,TOOHEY,2018,2018-03
4,4108,2,Detached House,15/11/2017,1/12/2017,2/01/2018,Public Housing,107,16,32,48,BRISBANE,TOOHEY,2018,2018-01
